In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
with open('communities.data') as file:
    lines = file.readlines()
    Data = []
    for l in lines:
        lread = l.split(',')[5:]
        Data.append(lread)      

In [3]:
with open('communities.names') as file:
    lines = file.readlines()[343:466]
    Name = []
    for l in lines:
        lread = l.split(' ')
        Name.append(lread[3])
    

In [4]:
for i in range(1994):
    for j in range(123):
        if Data[i][j] == '?':
            Data[i][j] = Name[j]
        Data[i][j] = float(Data[i][j])

In [5]:
""" Creating Training and Test Datasets for 5-fold Validation"""
"""first Data Sets"""
Test_1 = Data[:400]
Train_1 = Data[400:]

np.savetxt('CandC-Test(1).csv', Test_1, delimiter=',')
np.savetxt('CandC-Train(1).csv', Train_1, delimiter=',')

"""Second Data Sets"""
Test_2 = Data[400:799]
Train_2 = Data[799:]+Data[:400]

np.savetxt('CandC-Test(2).csv', Test_2, delimiter=',')
np.savetxt('CandC-Train(2).csv', Train_2, delimiter=',')

"""first Data Sets"""
Test_3 = Data [799:1198]
Train_3 = Data[1198:]+Data[:799]

np.savetxt('CandC-Test(3).csv', Test_3, delimiter=',')
np.savetxt('CandC-Train(3).csv', Train_3, delimiter=',')

"""first Data Sets"""
Test_4 = Data [1198:1597]
Train_4 = Data[1597:]+Data[:1198]

np.savetxt('CandC-Test(4).csv', Test_4, delimiter=',')
np.savetxt('CandC-Train(4).csv', Train_4, delimiter=',')

"""first Data Sets"""
Test_5 = Data [1597:]
Train_5 = Data[:1597]

np.savetxt('CandC-Test(5).csv', Test_5, delimiter=',')
np.savetxt('CandC-Train(5).csv', Train_5, delimiter=',')



In [6]:
""" Finding W_hat """
def calculate_w_hat (x_train , y_train):
    xtr = x_train.transpose()
    xtrx = np.dot(xtr , x_train)
    xtrx_inv = np.linalg.inv(xtrx)
    w1= np.dot(xtrx_inv , xtr)
    w_hat = np.dot(w1,y_train)
    return(w_hat)

In [7]:
""" Finding Y_hat """
def calculate_y_hat (x_train , w_hat):
    y_hat = np.dot(x_train , w_hat)
    return(y_hat)

In [8]:
""" Finding Mean_Square_Error (MSE) """
def calculate_MSE (y_hat, y_train):
    subtract = np.subtract(y_hat , y_train)
    squared = np.power(subtract, 2)
    add = squared.sum(axis = 0)
    MSE = np.divide(add, 50)
    return(MSE)

In [9]:
""" L2-Regularization """
def L2_Regularization(x_train , y_train, lam):
    i = np.identity(123)
    lam_i = np.dot( i, lam)
    xtr = x_train.transpose()
    xtrx = np.dot(xtr , x_train)
    w_1 = np.add(xtrx, lam_i )
    w_2 = np.linalg.inv (w_1)
    w_3 = np.dot(xtr , y_train)
    w_ridge = np.dot(w_2 , w_3)      
    return(w_ridge)

In [10]:
""" First Fold Train(1)  """
w_ridge_all_fold_1 = []
MSE_all_train_1 = []
MSE_all_test_1 = []
info = []
lambda_info_fold_1 = []

with open('CandC-Train(1).csv') as file:
    lines = file.readlines()
    Data_train_1 = []
    for l in lines:
        lread = l.split(',')
        for i in range(len(lread)):
            lread[i] = float(lread[i])
        Data_train_1.append(lread)  
    Data_train_1 = np.array(Data_train_1) 


Data_x_train_1 = Data_train_1 [:,:-1]
Data_x_train_1 = np.insert(Data_x_train_1, 0, 1, axis = 1) 
Data_y_train_1 = Data_train_1 [:,-1]


w_hat_train_1 = calculate_w_hat (Data_x_train_1 , Data_y_train_1)
y_hat_train_1 = calculate_y_hat (Data_x_train_1 , w_hat_train_1)
MSE_train_1 = calculate_MSE (y_hat_train_1, Data_y_train_1)

print('MSE_train_1:', MSE_train_1)

""" First Fold Test(1) """

with open('CandC-Test(1).csv') as file:
    lines = file.readlines()
    Data_test_1 = []
    for l in lines:
        lread = l.split(',')
        for i in range(len(lread)):
            lread[i] = float(lread[i])
        Data_test_1.append(lread)  
Data_test_1 = np.array(Data_test_1)        


Data_x_test_1 = Data_test_1 [:,:-1]
Data_x_test_1 = np.insert(Data_x_test_1, 0, 1, axis = 1)   
Data_y_test_1 = Data_test_1 [:,-1]


y_hat_test_1 = calculate_y_hat (Data_x_test_1 , w_hat_train_1)
MSE_test_1 = calculate_MSE (y_hat_test_1, Data_y_test_1)

print('MSE_test_1:', MSE_test_1)

""" Applying Ridge-Regression """

for lam in  np.linspace(0, 2, 4):
    """ Train Part Regularization """
    w_ridge = L2_Regularization(Data_x_train_1 , Data_y_train_1, lam)
    w_ridge_all_fold_1.append(w_ridge)
    y_hat_train_wridge = calculate_y_hat(Data_x_train_1 , w_ridge)
    MSE = calculate_MSE(y_hat_train_wridge , Data_y_train_1)
    MSE_all_train_1.append(MSE)
    """ Test part Regularization """
    y_hat_test_wridge = calculate_y_hat(Data_x_test_1 , w_ridge)
    MSE = calculate_MSE(y_hat_test_wridge , Data_y_test_1)
    MSE_all_test_1.append(MSE)
    lambda_info_fold_1.append([lam]) 

print ("Train MSE's after Regularization fold 1:", MSE_all_train_1)    
print ("Test MSE's after Regularization fold 1:", MSE_all_test_1)  


MSE_train_1: 0.498858846048
MSE_test_1: 0.162177086987
Train MSE's after Regularization fold 1: [0.49885884604782599, 0.50874716823995181, 0.51462531844253268, 0.51917698225023301]
Test MSE's after Regularization fold 1: [0.1621770869907215, 0.16056648217523026, 0.16039807030664086, 0.16049874624537053]


In [11]:
""" First Fold Train (2) """
w_ridge_all_fold_2 = []
MSE_all_train_2 = []
MSE_all_test_2 = []
lambda_info_fold_2 = []
with open('CandC-Train(2).csv') as file:
    lines = file.readlines()
    Data_train_2 = []
    for l in lines:
        lread = l.split(',')
        for i in range(len(lread)):
            lread[i] = float(lread[i])
        Data_train_2.append(lread)  
    Data_train_2 = np.array(Data_train_2) 


Data_x_train_2 = Data_train_2 [:,:-1]
Data_x_train_2 = np.insert(Data_x_train_2, 0, 1, axis = 1) 
Data_y_train_2 = Data_train_2 [:,-1]


w_hat_train_2 = calculate_w_hat (Data_x_train_2 , Data_y_train_2)
y_hat_train_2 = calculate_y_hat (Data_x_train_2 , w_hat_train_2)
MSE_train_2 = calculate_MSE (y_hat_train_2, Data_y_train_2)

print('MSE_train_2:', MSE_train_2)

""" First Fold Test (2) """

with open('CandC-Test(2).csv') as file:
    lines = file.readlines()
    Data_test_2 = []
    for l in lines:
        lread = l.split(',')
        for i in range(len(lread)):
            lread[i] = float(lread[i])
        Data_test_2.append(lread)  
Data_test_2 = np.array(Data_test_2)        


Data_x_test_2 = Data_test_2 [:,:-1]
Data_x_test_2 = np.insert(Data_x_test_2, 0, 1, axis = 1)   
Data_y_test_2 = Data_test_2 [:,-1]


y_hat_test_2 = calculate_y_hat (Data_x_test_2 , w_hat_train_2)
MSE_test_2 = calculate_MSE (y_hat_test_2, Data_y_test_2)

print('MSE_test_2:', MSE_test_2)

""" Applying Ridge-Regression """

for lam in  np.linspace(0, 2, 4):
    """ Train Part Regularization """
    w_ridge = L2_Regularization(Data_x_train_2 , Data_y_train_2, lam)
    w_ridge_all_fold_2.append(w_ridge)
    y_hat_train_wridge = calculate_y_hat(Data_x_train_2 , w_ridge)
    MSE = calculate_MSE(y_hat_train_wridge , Data_y_train_2)
    MSE_all_train_2.append(MSE)
    """ Test part Regularization """
    y_hat_test_wridge = calculate_y_hat(Data_x_test_2 , w_ridge)
    MSE = calculate_MSE(y_hat_test_wridge , Data_y_test_2)
    MSE_all_test_2.append(MSE)
    lambda_info_fold_2.append([lam]) 

print ("Train MSE's after Regularization fold 2:", MSE_all_train_2)    
print ("Test MSE's after Regularization fold 2:", MSE_all_test_2)  

MSE_train_2: 0.471393075317
MSE_test_2: 0.198352258795
Train MSE's after Regularization fold 2: [0.47139307531746494, 0.48652350171268938, 0.4932926858024787, 0.49806451698528392]
Test MSE's after Regularization fold 2: [0.19835225879780347, 0.18434034641548189, 0.18306161369832286, 0.1827545259887412]


In [12]:
""" First Fold Train (3) """
w_ridge_all_fold_3 = []
MSE_all_train_3 = []
MSE_all_test_3 = []
lambda_info_fold_3 = []

with open('CandC-Train(3).csv') as file:
    lines = file.readlines()
    Data_train_3 = []
    for l in lines:
        lread = l.split(',')
        for i in range(len(lread)):
            lread[i] = float(lread[i])
        Data_train_3.append(lread)  
    Data_train_3 = np.array(Data_train_3) 


Data_x_train_3 = Data_train_3 [:,:-1]
Data_x_train_3 = np.insert(Data_x_train_3, 0, 1, axis = 1) 
Data_y_train_3 = Data_train_3 [:,-1]


w_hat_train_3 = calculate_w_hat (Data_x_train_3 , Data_y_train_3)
y_hat_train_3 = calculate_y_hat (Data_x_train_3 , w_hat_train_3)
MSE_train_3 = calculate_MSE (y_hat_train_3, Data_y_train_3)

print('MSE_train_3:', MSE_train_3)

""" First Fold Test(3) """

with open('CandC-Test(3).csv') as file:
    lines = file.readlines()
    Data_test_3 = []
    for l in lines:
        lread = l.split(',')
        for i in range(len(lread)):
            lread[i] = float(lread[i])
        Data_test_3.append(lread)  
Data_test_3 = np.array(Data_test_3)        


Data_x_test_3 = Data_test_3 [:,:-1]
Data_x_test_3 = np.insert(Data_x_test_3, 0, 1, axis = 1)   
Data_y_test_3 = Data_test_3 [:,-1]


y_hat_test_3 = calculate_y_hat (Data_x_test_3 , w_hat_train_3)
MSE_test_3 = calculate_MSE (y_hat_test_3, Data_y_test_3)

print('MSE_test_3:', MSE_test_3)

""" Applying Ridge-Regression """

for lam in  np.linspace(0, 2, 4):
    """ Train Part Regularization """
    w_ridge = L2_Regularization(Data_x_train_3 , Data_y_train_3, lam)
    w_ridge_all_fold_3.append(w_ridge)
    y_hat_train_wridge = calculate_y_hat(Data_x_train_3 , w_ridge)
    MSE = calculate_MSE(y_hat_train_wridge , Data_y_train_3)
    MSE_all_train_3.append(MSE)
    """ Test part Regularization """
    y_hat_test_wridge = calculate_y_hat(Data_x_test_3 , w_ridge)
    MSE = calculate_MSE(y_hat_test_wridge , Data_y_test_3)
    MSE_all_test_3.append(MSE)
    lambda_info_fold_3.append([lam]) 

print ("Train MSE's after Regularization fold 3:", MSE_all_train_3)    
print ("Test MSE's after Regularization fold 3:", MSE_all_test_3) 

MSE_train_3: 0.527666970381
MSE_test_3: 0.136387004567
Train MSE's after Regularization fold 3: [0.52766697038085464, 0.54046486486511203, 0.54645440368905718, 0.55079955225342336]
Test MSE's after Regularization fold 3: [0.13638700457476954, 0.13031629032638783, 0.13061245473559807, 0.13108506157467356]


In [13]:
""" First Fold Train (4) """
w_ridge_all_fold_4 = []
MSE_all_train_4 = []
MSE_all_test_4 = []
lambda_info_fold_4 = []
with open('CandC-Train(4).csv') as file:
    lines = file.readlines()
    Data_train_4 = []
    for l in lines:
        lread = l.split(',')
        for i in range(len(lread)):
            lread[i] = float(lread[i])
        Data_train_4.append(lread)  
    Data_train_4 = np.array(Data_train_4) 


Data_x_train_4 = Data_train_4 [:,:-1]
Data_x_train_4 = np.insert(Data_x_train_4, 0, 1, axis = 1) 
Data_y_train_4 = Data_train_4 [:,-1]


w_hat_train_4 = calculate_w_hat (Data_x_train_4 , Data_y_train_4)
y_hat_train_4 = calculate_y_hat (Data_x_train_4 , w_hat_train_4)
MSE_train_4 = calculate_MSE (y_hat_train_4, Data_y_train_4)

print('MSE_train_4:', MSE_train_4)

""" First Fold Test(4) """

with open('CandC-Test(4).csv') as file:
    lines = file.readlines()
    Data_test_4 = []
    for l in lines:
        lread = l.split(',')
        for i in range(len(lread)):
            lread[i] = float(lread[i])
        Data_test_4.append(lread)  
Data_test_4 = np.array(Data_test_4)        


Data_x_test_4 = Data_test_4 [:,:-1]
Data_x_test_4 = np.insert(Data_x_test_4, 0, 1, axis = 1)   
Data_y_test_4 = Data_test_4 [:,-1]


y_hat_test_4 = calculate_y_hat (Data_x_test_4 , w_hat_train_4)
MSE_test_4 = calculate_MSE (y_hat_test_4, Data_y_test_4)

print('MSE_test_4:', MSE_test_4)

""" Applying Ridge-Regression """

for lam in  np.linspace(0, 2, 4):
    """ Train Part Regularization """
    w_ridge = L2_Regularization(Data_x_train_4 , Data_y_train_4, lam)
    w_ridge_all_fold_4.append(w_ridge)
    y_hat_train_wridge = calculate_y_hat(Data_x_train_4 , w_ridge)
    MSE = calculate_MSE(y_hat_train_wridge , Data_y_train_4)
    MSE_all_train_4.append(MSE)
    """ Test part Regularization """
    y_hat_test_wridge = calculate_y_hat(Data_x_test_4 , w_ridge)
    MSE = calculate_MSE(y_hat_test_wridge , Data_y_test_4)
    MSE_all_test_4.append(MSE)
    lambda_info_fold_4.append([lam]) 

print ("Train MSE's after Regularization fold 4:", MSE_all_train_4)    
print ("Test MSE's after Regularization fold 4:", MSE_all_test_4) 


MSE_train_4: 0.532703224176
MSE_test_4: 0.129859562775
Train MSE's after Regularization fold 4: [0.5327032241762929, 0.5428107908296671, 0.54833700057206836, 0.55262949354656354]
Test MSE's after Regularization fold 4: [0.12985956277478281, 0.1274037669115233, 0.12721871326927323, 0.12714367544923494]


In [14]:
""" First Fold Train (5) """
w_ridge_all_fold_5 = []
MSE_all_train_5 = []
MSE_all_test_5 = []
lambda_info_fold_5 = []
with open('CandC-Train(5).csv') as file:
    lines = file.readlines()
    Data_train_5 = []
    for l in lines:
        lread = l.split(',')
        for i in range(len(lread)):
            lread[i] = float(lread[i])
        Data_train_5.append(lread)  
    Data_train_5 = np.array(Data_train_5) 


Data_x_train_5 = Data_train_5 [:,:-1]
Data_x_train_5 = np.insert(Data_x_train_5, 0, 1, axis = 1) 
Data_y_train_5 = Data_train_5 [:,-1]


w_hat_train_5 = calculate_w_hat (Data_x_train_5 , Data_y_train_5)
y_hat_train_5 = calculate_y_hat (Data_x_train_5 , w_hat_train_5)
MSE_train_5 = calculate_MSE (y_hat_train_5, Data_y_train_5)

print('MSE_train_5:', MSE_train_5)

""" First Fold Test(5) """

with open('CandC-Test(5).csv') as file:
    lines = file.readlines()
    Data_test_5 = []
    for l in lines:
        lread = l.split(',')
        for i in range(len(lread)):
            lread[i] = float(lread[i])
        Data_test_5.append(lread)  
Data_test_5 = np.array(Data_test_5)        


Data_x_test_5 = Data_test_5 [:,:-1]
Data_x_test_5 = np.insert(Data_x_test_5, 0, 1, axis = 1)   
Data_y_test_5 = Data_test_5 [:,-1]


y_hat_test_5 = calculate_y_hat (Data_x_test_5 , w_hat_train_5)
MSE_test_5 = calculate_MSE (y_hat_test_5, Data_y_test_5)

print('MSE_test_5:', MSE_test_5)

""" Applying Ridge-Regression """

for lam in  np.linspace(0, 2, 4):
    """ Train Part Regularization """
    w_ridge = L2_Regularization(Data_x_train_5 , Data_y_train_5, lam)
    w_ridge_all_fold_5.append(w_ridge)
    y_hat_train_wridge = calculate_y_hat(Data_x_train_5 , w_ridge)
    MSE = calculate_MSE(y_hat_train_wridge , Data_y_train_5)
    MSE_all_train_5.append(MSE)
    """ Test part Regularization """
    y_hat_test_wridge = calculate_y_hat(Data_x_test_5 , w_ridge)
    MSE = calculate_MSE(y_hat_test_wridge , Data_y_test_5)
    MSE_all_test_5.append(MSE)
    lambda_info_fold_5.append([lam]) 

print ("Train MSE's after Regularization fold 5:", MSE_all_train_5)    
print ("Test MSE's after Regularization fold 5:", MSE_all_test_5) 


MSE_train_5: 2.83741335748e+32
MSE_test_5: 7.86373444933e+31
Train MSE's after Regularization fold 5: [1.3429189739767096e+33, 0.52687847331204474, 0.53308830201008273, 0.53797488113939074]
Test MSE's after Regularization fold 5: [3.4703436948057319e+32, 0.14369496295043077, 0.14242146170811626, 0.14165333467652996]


In [15]:
""" Report : 5-fold Cross Validation Error """
MSE_all_tests = []
MSE_test_after_regul = []
MSE_train_after_regul = []
info = []
# lambda_info_fold_5 = np.matrix(lambda_info_fold_5, dtype= np.float)
# MSE_all_test_5 = np.matrix(MSE_all_test_5, dtype= np.float)
# print (lambda_info_fold_5 , MSE_all_test_5)
# # info.append([lambda_info_fold_5,'Average MSE Test:' , MSE_test_3, MSE_test_4,MSE_test_5])
# # for i in info:
# #     print(i)


MSE_all_tests = np.append(MSE_all_tests , MSE_test_1)
MSE_all_tests = np.append(MSE_all_tests , MSE_test_2)
MSE_all_tests = np.append(MSE_all_tests , MSE_test_3)
MSE_all_tests = np.append(MSE_all_tests , MSE_test_4)
MSE_all_tests = np.append(MSE_all_tests , MSE_test_5)


MSE_mean = MSE_all_tests.mean()
print('Average MSE Tests:',MSE_mean)
print('\n''MSE_train_1:', MSE_train_1)
print('\n''MSE_test_1:', MSE_test_1)
print('\n''Parameter Learnt for fold 1 (w_hat_train_1):\n', w_hat_train_1)
print('-----------------------------')
print('\n''MSE_train_2:', MSE_train_2)
print('\n''MSE_test_2:', MSE_test_2)
print('\n''Parameter Learnt for fold 2 (w_hat_train_2):\n', w_hat_train_2)
print('-----------------------------')
print('\n''MSE_train_3:', MSE_train_3)
print('\n''MSE_test_3:', MSE_test_3)
print('\n''Parameter Learnt for fold 3 (w_hat_train_3):\n', w_hat_train_3)
print('-----------------------------')
print('\n''MSE_train_4:', MSE_train_4)
print('\n''MSE_test_4:', MSE_test_4)
print('\n''Parameter Learnt for fold 4 (w_hat_train_4):\n', w_hat_train_4)
print('-----------------------------')
print('\n''MSE_train_5:', MSE_train_5)
print('\n''MSE_test_5:', MSE_test_5)
print('\n''Parameter Learnt for fold 5 (w_hat_train_5):\n', w_hat_train_5)
print('-----------------------------')

Average MSE Tests: 1.57274688987e+31

MSE_train_1: 0.498858846048

MSE_test_1: 0.162177086987

Parameter Learnt for fold 1 (w_hat_train_1):
 [  1.45678405e+00   9.46384253e-02  -4.80307711e-02   1.90335106e-01
  -3.44353053e-02  -3.41911780e-02   8.24132431e-02   1.14492023e-01
  -1.44615898e-01  -1.78738550e-01   2.86439565e-02  -3.30648697e-01
   5.88503484e-02  -1.98377845e-01  -2.20875332e-01   3.17132445e-02
  -1.59109475e-01   1.28154578e-01   1.86026386e-02  -1.10962008e-01
   2.45194269e-01   4.00757987e-02  -3.87242297e-01  -3.02266116e-02
  -2.79508952e-02   2.40945222e-02   4.10437629e-02   2.97141850e-02
   1.35376123e-01  -2.28770839e-01  -1.19992737e-01   9.84379607e-02
   1.25532417e-01   5.52706313e-02   2.57197204e-01  -8.09062076e-02
  -5.95424709e-02   9.74870911e-02   1.54964829e-01   3.43774106e-01
   1.94121408e-01   6.56941601e-03  -3.45755178e-01  -2.50443261e-01
   4.29464463e-02  -3.79697978e-01  -2.41638477e-03   2.52162302e-03
   6.16463855e-02  -1.68881087e

In [16]:
MSE_Test = []
lambda_info_all = np.array(lambda_info_fold_5)
MSE_Test = np.matrix(MSE_Test)
MSE_Test = np.insert(MSE_Test, 0, lambda_info_fold_5, axis = 1) 

MSE_Test_1 =  np.array(MSE_all_test_1)
MSE_Test = np.insert(MSE_Test, 1, MSE_Test_1, axis = 0) 


MSE_Test_2 =  np.array(MSE_all_test_2)
MSE_Test = np.insert(MSE_Test, 2, MSE_Test_2,  axis = 0) 

MSE_Test_3 =  np.array(MSE_all_test_3)
MSE_Test = np.insert(MSE_Test, 3, MSE_Test_3,  axis = 0) 

MSE_Test_4 =  np.array(MSE_all_test_4)
MSE_Test = np.insert(MSE_Test, 4, MSE_Test_4,  axis = 0) 

MSE_Test_5 =  np.array(MSE_all_test_5)
MSE_Test = np.insert(MSE_Test, 5, MSE_Test_5,  axis = 0) 

MSE_Test_All = MSE_Test[:][1]+MSE_Test[:][2]+MSE_Test[:][3]+MSE_Test[:][4]+MSE_Test[:][5]

print('Different Values of Lambda:          ', MSE_Test[0][:], )
print('Test MSE fold 1 Correspond to Lambda:',MSE_Test[1][:])
print( 'Test MSE fold 2 Correspond to Lambda:',MSE_Test[2][:])
print( 'Test MSE fold 3 Correspond to Lambda:',MSE_Test[3][:])
print( 'Test MSE fold 4 Correspond to Lambda:',MSE_Test[4][:])
print( 'Test MSE fold 5 Correspond to Lambda:',MSE_Test[5][:])
print('\n')
print ('Average MSE Test (for Lambda = 0) :         ', np.divide(MSE_Test_All[0,0], 4))
print ('Average MSE Test (for Lambda = 0.66666667) :', np.divide(MSE_Test_All[0,1], 4))
print ('Average MSE Test (for Lambda = 1.33333333) :', np.divide(MSE_Test_All[0,2], 4))
print ('Average MSE Test (for Lambda = 2) :         ', np.divide(MSE_Test_All[0,3], 4))



print('\n''Parameter Learnt for fold 1 (w_ridge):\n ',w_ridge_all_fold_1)
print('\n''Parameter Learnt for fold 2 (w_ridge):\n ',w_ridge_all_fold_2)
print('\n''Parameter Learnt for fold 3 (w_ridge):\n ',w_ridge_all_fold_3)
print('\n''Parameter Learnt for fold 4 (w_ridge):\n ',w_ridge_all_fold_4)
print('\n''Parameter Learnt for fold 5 (w_ridge):\n ',w_ridge_all_fold_5)



Different Values of Lambda:           [[ 0.          0.66666667  1.33333333  2.        ]]
Test MSE fold 1 Correspond to Lambda: [[ 0.16217709  0.16056648  0.16039807  0.16049875]]
Test MSE fold 2 Correspond to Lambda: [[ 0.19835226  0.18434035  0.18306161  0.18275453]]
Test MSE fold 3 Correspond to Lambda: [[ 0.136387    0.13031629  0.13061245  0.13108506]]
Test MSE fold 4 Correspond to Lambda: [[ 0.12985956  0.12740377  0.12721871  0.12714368]]
Test MSE fold 5 Correspond to Lambda: [[  3.47034369e+32   1.43694963e-01   1.42421462e-01   1.41653335e-01]]


Average MSE Test (for Lambda = 0) :          8.67585923701e+31
Average MSE Test (for Lambda = 0.66666667) : 0.186580462195
Average MSE Test (for Lambda = 1.33333333) : 0.185928078429
Average MSE Test (for Lambda = 2) :          0.185783835984

Parameter Learnt for fold 1 (w_ridge):
  [array([  1.45678405e+00,   9.46384253e-02,  -4.80307711e-02,
         1.90335106e-01,  -3.44353053e-02,  -3.41911780e-02,
         8.24132431e-02,   1.1

In [17]:
np.linspace(0, 2, 4)

array([ 0.        ,  0.66666667,  1.33333333,  2.        ])